#  Analysis on IPL dataset - part 4

(Based on IPL matches and deliveries dataset)


Matches- <a>https://www.kaggle.com/nowke9/ipldata#matches.csv</a>


Deliveries- <a>https://www.kaggle.com/nowke9/ipldata#deliveries.csv<a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
match=pd.read_csv('/content/matches.csv')
delivery=pd.read_csv('/content/deliveries.csv')

In [0]:
match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 18 columns):
id                 756 non-null int64
season             756 non-null int64
city               749 non-null object
date               756 non-null object
team1              756 non-null object
team2              756 non-null object
toss_winner        756 non-null object
toss_decision      756 non-null object
result             756 non-null object
dl_applied         756 non-null int64
winner             752 non-null object
win_by_runs        756 non-null int64
win_by_wickets     756 non-null int64
player_of_match    752 non-null object
venue              756 non-null object
umpire1            754 non-null object
umpire2            754 non-null object
umpire3            119 non-null object
dtypes: int64(5), object(13)
memory usage: 106.4+ KB


In [0]:
delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179078 entries, 0 to 179077
Data columns (total 21 columns):
match_id            179078 non-null int64
inning              179078 non-null int64
batting_team        179078 non-null object
bowling_team        179078 non-null object
over                179078 non-null int64
ball                179078 non-null int64
batsman             179078 non-null object
non_striker         179078 non-null object
bowler              179078 non-null object
is_super_over       179078 non-null int64
wide_runs           179078 non-null int64
bye_runs            179078 non-null int64
legbye_runs         179078 non-null int64
noball_runs         179078 non-null int64
penalty_runs        179078 non-null int64
batsman_runs        179078 non-null int64
extra_runs          179078 non-null int64
total_runs          179078 non-null int64
player_dismissed    8834 non-null object
dismissal_kind      8834 non-null object
fielder             6448 non-null object
dtype

In [0]:
df=match.merge(delivery,left_on='id',right_on='match_id')

In [0]:
#  Q1.- Make a pivot table of top 10 bowlers(in terms of wickets) with over(1-20) showing no. of wickets.  [Dificult-]

In [0]:
bowlers=delivery[delivery['dismissal_kind'].isin(['caught', 'bowled', 'lbw', 'caught and bowled','stumped'])].groupby('bowler').size().sort_values(ascending=False).head(10).reset_index()['bowler'].tolist()
delivery[(delivery['bowler'].isin(bowlers)) & (delivery['dismissal_kind'].isin(['caught', 'bowled', 'lbw', 'caught and bowled','stumped']))].pivot_table(index='bowler',columns='over',values='dismissal_kind',fill_value=0,aggfunc='count')

over,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
bowler,,,,,,,,,,,,,,,,,,,,
A Mishra,1,0,0,4,1,0,15,9,20,10,13,13,10,16,6,14,7,6,5,6
B Kumar,17,7,12,3,4,3,0,3,0,0,1,1,1,1,2,12,8,16,19,23
DJ Bravo,0,1,2,2,1,4,7,4,6,2,3,6,7,7,5,13,9,29,10,29
Harbhajan Singh,5,5,1,7,4,8,12,13,8,12,12,13,6,12,5,7,8,6,4,1
PP Chawla,0,2,1,1,2,10,6,8,10,9,15,8,15,11,14,11,11,6,2,7
R Ashwin,7,1,5,5,9,8,2,9,5,5,10,10,7,7,7,15,4,7,1,1
RA Jadeja,0,0,0,0,2,4,10,4,9,9,7,8,7,9,13,3,11,5,4,3
SL Malinga,11,11,2,3,5,5,1,0,1,2,1,4,9,3,4,18,14,25,32,19
SP Narine,0,2,2,3,9,8,0,3,4,2,10,5,4,6,7,9,12,13,12,11


In [0]:
# Q2.- Make a function for batting profile of any batsman:-                                              [Medium+]



Ex- 

V Kohli:

Innings:  

Total Runs: 

No. of MOM(Man of the match):

No. of boundries:

In [0]:
def batting_profile(batsman_name):
    batsman=df[df['batsman']==batsman_name]
    
    mom=batsman[batsman['player_of_match']==batsman_name].groupby(['match_id','player_of_match']).size().reset_index()['player_of_match'].count()
    innings=batsman.groupby(['match_id','inning']).size().reset_index()['inning'].count()
    runs=batsman['batsman_runs'].sum()
    boundaries=batsman[batsman['batsman_runs'].isin([4,6])]['batsman_runs'].count()
    
    return(pd.DataFrame({"Batsman": [batsman_name],"Innings": innings,"Total_Runs": runs,"No. of MOM": mom,'No. of Boundaries':boundaries}))

In [0]:
batting_profile('V Kohli')

,Batsman,Innings,Total_Runs,No. of MOM,No. of Boundaries
0,V Kohli,170,5434,12,673


In [0]:
batting_profile('CH Gayle')

,Batsman,Innings,Total_Runs,No. of MOM,No. of Boundaries
0,CH Gayle,127,4560,21,703


In [0]:
# Q3.- Write a function to return top-4 batsman at a venue. Take venue as input.

In [0]:
def top_batsman_in(venue):
    return df[df['venue']==venue].groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(4).reset_index()

In [0]:
top_batsman_in('Eden Gardens')

,batsman,batsman_runs
0,G Gambhir,1407
1,RV Uthappa,1178
2,YK Pathan,863
3,AD Russell,719


In [0]:
top_batsman_in('M Chinnaswamy Stadium')

,batsman,batsman_runs
0,V Kohli,2114
1,AB de Villiers,1712
2,CH Gayle,1540
3,Mandeep Singh,419


In [0]:
# Q4.- No. of matches per season played in Mumbai.

In [0]:
match[match['city']=='Mumbai'].groupby('season').size().reset_index().rename(columns={0:'matches_played'})

,season,matches_played
0,2008,10
1,2010,13
2,2011,16
3,2012,8
4,2013,8
5,2014,7
6,2015,11
7,2016,4
8,2017,8
9,2018,9


In [0]:
# Q5. - Top-3 Player in terms of most no. of catches.

In [0]:
delivery[delivery['dismissal_kind'].isin(['caught'])].groupby('fielder').size().sort_values(ascending=False).head(3).reset_index().rename(columns={0:'catches'})

,fielder,catches
0,KD Karthik,109
1,SK Raina,99
2,MS Dhoni,98
